In [ ]:
# 短时memory 记忆

In [6]:
from src.llm.llm_factory import LLMFactory

llm = LLMFactory.create_llm()

In [1]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("你好，我是人类！")
memory.chat_memory.add_ai_message("你好，我是AI，有什么可以帮助你的吗？")
memory.load_memory_variables({})

C:\Users\longyu3\AppData\Local\Temp\ipykernel_26328\2725643520.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


{'history': 'Human: 你好，我是人类！\nAI: 你好，我是AI，有什么可以帮助你的吗？'}

In [4]:
# 超过指定条数，会被删除掉
# 实现一个最近的对话窗口，超过窗口条数的对话将被删除
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)

memory.save_context({"input":"你好，我是人类！"}, {"output":"你好，我是AI，有什么可以帮助你的吗？"})
memory.save_context({"input":"我想吃鸡肉"}, {"output":"好的，我帮你找找鸡肉的做法"})

memory.load_memory_variables({})

{'history': 'Human: 我想吃鸡肉\nAI: 好的，我帮你找找鸡肉的做法'}

In [5]:
# 构建记忆实体概念清单, 相当于把你们的信息 剥离抽离成一个个单元实体
from langchain.llms import OpenAI
from langchain.memory import ConversationEntityMemory
memory = ConversationEntityMemory(llm=llm)
_input = {
    "input":"胡八一和王胖子雪莉杨经常在一起冒险，合称盗墓铁三角。"
}
memory.load_memory_variables(_input)


In [7]:
# 使用知识图谱构建记忆模块
from langchain.memory import ConversationKGMemory
memory = ConversationKGMemory(llm=llm)
memory.save_context(
    {"input":"帮我找一下tomie"},
    {"output":"对不起请问什么是tomie？"}
)

memory.save_context(
    {"input": "tomie是一个培训讲师"},
    {"output": "好的，我知道了。"}
)

In [8]:
# 长对话在内存中的处理，对对话内容进行摘要

from langchain.memory import ConversationSummaryMemory
memory = ConversationSummaryMemory(llm=llm)
memory.save_context(
    {"input":"帮我找一下tomie"},
    {"output":"对不起请问什么是tomie？"}
)
memory.save_context(
    {"input":"tomie是一个培训讲师"},
    {"output":"好的，我知道了。"}
)


C:\Users\longyu3\AppData\Local\Temp\ipykernel_26328\2208959937.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(llm=llm)


In [9]:
messages = memory.chat_memory.messages
# print(messages)
memory.predict_new_summary(messages, "")

'The human asks the AI to help find "tomie." The AI initially doesn\'t understand what "tomie" is, but after the human explains that it\'s a training instructor, the AI acknowledges the information.'

In [ ]:
# 使用chathistory 快速获得总结
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()
history.add_user_message("你好，我是人类！")
history.add_ai_message("你好，我是AI小丸子，有什么可以帮助你的吗？")

memory = ConversationSummaryMemory.from_messages(
    llm=llm,
    chat_memory=history,
    return_messages=True
)

In [ ]:
memory.buffer

In [ ]:
# 长对话进行summary
from langchain.memory import ConversationSummaryBufferMemory
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=10,
    return_messages=True
)
memory.save_context(
    {"input":"帮我找一下tomie"},
    {"output":"对不起请问什么是tomie？"}
)
memory.save_context(
    {"input":"tomie是一个培训讲师"},
    {"output":"好的，我知道了。"}
)
memory.save_context(
    {"input":"今天他要讲一门关于RAG的课程"},
    {"output":"好的，我知道了。需要RAG的资料吗？"}
)



In [ ]:
memory.load_memory_variables({})

In [ ]:
# Conversation Token Buffer使用token长度来决定什么时候刷新内存
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI
memory = ConversationTokenBufferMemory(
    llm=llm,
    max_token_limit=100  # 假设设置的最大token限制为100
)
memory.save_context(
    {"input":"帮我找一下tomie"},
    {"output":"对不起请问什么是tomie？"}
)
memory.save_context(
    {"input":"tomie是一个培训讲师"},
    {"output":"好的，我知道了。"}
)
memory.save_context(
    {"input":"今天他要讲一门关于RAG的课程"},
    {"output":"好的，我知道了。需要RAG的资料吗？"}
)
memory.load_memory_variables({})


# 在Chain上面使用记忆

In [10]:
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

In [ ]:
# 自定义模板
template = """你是一个可以和人类对话的机器人。
{chat_history}
人类:{human_input}
机器人:"""

prompt = PromptTemplate(
    template=template,
    input_variables=["chat_history", "human_input"],
)

memory = ConversationBufferMemory(
    memory_key="chat_history",
)
chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True,
)

In [ ]:
# 这里就可以循环对话，自动存储memory了
chain.predict(human_input="你好，我是人类！")

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain.schema import SystemMessage, HumanMessage

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="你好，我是一个可以和人类对话的机器人",
        ),
        MessagesPlaceholder(
            variable_name="chat_history",
        ),
        HumanMessagePromptTemplate.from_template(
            "{human_input}",
        ),
    ]
)
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
)
chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True,
)
chain.predict(human_input="你好")

In [ ]:
# Conversation Chain 中增加记忆
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

memory = ConversationBufferMemory(
    memory_key = "history",
    return_messages=True
)
# 给对话链增加记忆
chain = ConversationChain(
    llm = llm,
    memory = memory,
    verbose=True
)

chain.predict(input="你好，我是人类！")


In [ ]:
# 自定义一下，对其进行覆盖
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

template = """下面是一段AI与人类的对话，AI会针对人类问题提供尽可能详细的回答，如果AI不知道答案，会直接回复'人类老爷，我真的不知道'。
当前对话：
{history}
Human:{input}
AI助手:"""

prompt = PromptTemplate(
    template=template,
    input_variables=["history", "input"],
)
ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory(
        ai_prefix = "AI助手",
        return_messages=True
    ),
    prompt=prompt,
    verbose=True
)

# 同一个链合并使用多个记忆

In [ ]:
from langchain.chains import ConversationChain
from langchain.llms import OpenAI
from langchain.memory import (
    ConversationBufferMemory,
    ConversationSummaryMemory,
    CombinedMemory
)
from langchain.prompts import PromptTemplate
# 使用ConversationSummaryMemory对对话进行总结
summary = ConversationSummaryMemory(
    llm=llm,
    memory_key="history",
    input_key="input"
)

# 使用ConversationBufferMemory对对话进行缓存
cov_memory = ConversationBufferMemory(
    memory_key="history_now",
    input_key="input",
)
combined_memory = CombinedMemory(
    memories=[summary, cov_memory]
)

TEMPLATE = """下面是一段AI与人类的对话，AI会针对人类问题，提供之前的对话摘要：
{history}
当前对话：
{history_now}
Human:{input}
AI:"""

PromptTemplate(
    template=TEMPLATE,
    input_variables=["history", "history_now", "input"]
)
chain = ConversationChain(
    llm=llm,
    memory=combined_memory,
    prompt=prompt,
    verbose=True,
)
chain.run("你对加密货币市场怎么看")

In [ ]:
# 多参数链 增加记忆

